## Import necessary libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.spatial import cKDTree
from scipy.ndimage import binary_erosion

import cv2
from google.colab.patches import cv2_imshow

## Extract wound boundary

In [ ]:
def extract_boundary(mask: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """
    Extract boundary pixels from a binary mask.

    Parameters
    ----------
    mask : np.ndarray
        A 2D array representing a binary segmentation mask.
        - Expected dtype: boolean or integer (0/1)
        - Expected shape: (H, W)
        - If mask is not boolean, values != 0 are treated as foreground.

    Returns
    -------
    pts : np.ndarray
        Array of coordinates of boundary pixels with shape (N, 2).
        Each row is (y, x) where:
          - y is the row index (0..H-1)
          - x is the column index (0..W-1)
        Order of points is determined by np.where (row-major; not geometrically ordered).

    boundary : np.ndarray
        A 2D boolean array with the same shape as `mask`. True where a pixel is a
        boundary pixel (foreground AND was removed by erosion), False elsewhere.

    Notes
    -----
    - The function uses morphological erosion to remove interior pixels; the XOR
      between the original mask and the eroded mask leaves only the boundary.
    - Uses scipy.ndimage.binary_erosion with default structuring element
      (connectivity 1 / 3x3 cross). You can replace or wrap this if different
      connectivity is desired.
    - Coordinates are returned as (y, x) to match NumPy row/col indexing.
    - If there are no boundary pixels (e.g., empty mask), `pts` will be an array
      with shape (0, 2) and `boundary` will be all-False.
  """

  # Normalize to boolean: treat any non-zero as foreground
  mask_bool = mask.astype(bool)

  # Erode the foreground (pixels at the edge will likely become false)
  eroded = binary_erosion(mask)
  boundary = mask ^ eroded # XOR -> True for pixels present in mask but not in eroded

  # Convert boolean mask to a list of (row, col) coords -> (y, x)
  pts = np.column_stack(np.where(boundary))

  return pts, boundary

### Hausdorff Distance = the largest of all the closest distances between the two boundaries.

In [ ]:
def hausdorff_distance_optimized(mask_pred: np.ndarray, mask_gt: np.ndarray, pixel_spacing=(1.0, 1.0)) -> float:
    """
    Compute symmetric Hausdorff Distance efficiently using KD-Tree to avoid large memory usage.
    """
    # Extract boundary points
    A_pts, _ = extract_boundary(mask_pred)
    B_pts, _ = extract_boundary(mask_gt)

    # Apply pixel spacing
    A_pts = A_pts * np.array(pixel_spacing)
    B_pts = B_pts * np.array(pixel_spacing)

    # Build KD-Trees
    tree_A = cKDTree(A_pts)
    tree_B = cKDTree(B_pts)

    # Forward: A -> B
    hd_forward = np.max(tree_B.query(A_pts)[0])

    # Backward: B -> A
    hd_backward = np.max(tree_A.query(B_pts)[0])

    # Symmetric Hausdorff Distance
    return max(hd_forward, hd_backward)

## Average Symmetric Surface Distance (ASSD) = Average of all boundary mismatches

In [ ]:
def assd_optimized(mask_pred: np.ndarray, mask_gt: np.ndarray, pixel_spacing: tuple[float, float]=(1.0, 1.0), max_points: int = 2000) -> float:
    """
    Compute the Average Symmetric Surface Distance (ASSD) efficiently using KD-Trees.

    ASSD measures the average distance between the surface (boundary) points of two binary masks.
    It is symmetric, meaning it considers distances from prediction to ground-truth and vice versa.

    This optimized version avoids creating the full pairwise distance matrix, making it suitable
    for high-resolution images.

    Args:
        mask_pred (np.ndarray): Binary mask (H, W) of the predicted segmentation.
        mask_gt (np.ndarray): Binary mask (H, W) of the ground-truth segmentation.
        pixel_spacing (tuple[float, float], optional): Spacing of pixels along (y, x) axes in real-world units.
                                                       Defaults to (1.0, 1.0).
        max_points (int, optional): Maximum number of boundary points to use per mask. Randomly sampled
                                    if exceeded. Defaults to 2000.

    Returns:
        float: Average symmetric surface distance between the masks in the same units as `pixel_spacing`.
               Lower values indicate better agreement between predicted and ground-truth masks.
    """
    # Extract boundary points
    A_pts, _ = extract_boundary(mask_pred)
    B_pts, _ = extract_boundary(mask_gt)

    # Apply pixel spacing
    A_pts = A_pts * np.array(pixel_spacing)
    B_pts = B_pts * np.array(pixel_spacing)

    # Build KD-Trees
    tree_A = cKDTree(A_pts)
    tree_B = cKDTree(B_pts)

    # Compute nearest neighbor distances
    dist_A_to_B = tree_B.query(A_pts)[0]  # distances from A -> B
    dist_B_to_A = tree_A.query(B_pts)[0]  # distances from B -> A

    # Average symmetric surface distance
    return (dist_A_to_B.mean() + dist_B_to_A.mean()) / 2

## Helper functions

In [ ]:
def get_wound_predicted_gt_cleaned_paths(wound_base_path, wound_list):
  wound_data = []


  for item in wound_list:
    for wound_id, _ in item.items():
      wound_path = f"{wound_base_path}/wound/wound{wound_id}.jpg"
      predicted_path = f"{wound_base_path}/predicted/wound{wound_id}_mask.webp"
      gt_path = f"{wound_base_path}/groundtruth/wound{wound_id}.jpg"

      wound_data.append({
          "wound_id": wound_id,
          "wound_path": wound_path,
          "predicted_path": predicted_path,
          "gt_path": gt_path,
      })

  return wound_data

In [ ]:
def get_ruler_paths(ruler_base_path, wound_list):
  ruler_data = []

  for item in wound_list:
    for wound_id, real_length_mm in item.items():
      ruler_path = f"{ruler_base_path}/wound{wound_id}.jpg"

      ruler_data.append({
          "wound_id": wound_id,
          "ruler_path": ruler_path,
          "real_length_mm": real_length_mm,
      })

  return ruler_data

In [ ]:
def order_rectangle_points(pts):
    """
    Take 4 unordered points of a rectangle and order them in TL, TR, BR, BL order"""

    # Computes x+y for each point (TL will have the smallest, BR will have the largest)
    s = pts.sum(axis=1)
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    # Compute y-x for each point (TR will have the smallest, BL will have the largest)
    diff = np.diff(pts, axis=1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl])

def calibrate_ruler_convex_hull_grid(ruler_path, real_length_mm, white_thresh=245):

    # Load the ruler image path
    img = cv2.imread(ruler_path)
    if img is None:
        raise FileNotFoundError(f"Cannot load: {ruler_path}")

    h, w = img.shape[:2]

    # Extract Ruler Mask
    mask = np.any(img < white_thresh, axis=2).astype(np.uint8)
    kernel = np.ones((10, 10), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Extract Coordinates
    ys, xs = np.where(mask == 1)
    coords = np.column_stack((xs, ys)).astype(np.int32)
    if len(coords) < 2:
        raise ValueError("Insufficient ruler pixels found.")

    # Convex Hull (Compute the smallest convex polygon enclosing all ruler pixels)
    hull = cv2.convexHull(coords)

    # Allow simplification within 2% of perimeter
    epsilon = 0.02 * cv2.arcLength(hull, True)

    # Approximate the hull with fewer points (4 corners of the ruler in our case)
    approx = cv2.approxPolyDP(hull, epsilon, True)
    if len(approx) != 4:
        raise ValueError("Could not find 4 corners. Try adjusting epsilon.")

    # corner is in (num_points, 1, 2), we only need (num_points, 2)
    corners = approx.reshape(4, 2)
    corners = order_rectangle_points(corners)
    tl, tr, br, bl = corners

    # Compute the Euclidean distance of the rectangle sides
    width = np.linalg.norm(tr - tl)
    height = np.linalg.norm(bl - tl)
    longest_edge = max(width, height)
    pixel_spacing = real_length_mm / longest_edge

    print("Longest edge (ruler):", longest_edge, "pixels")
    print("Pixel spacing:", pixel_spacing, "mm per pixel")

    # Ruler image with 4 points and longest side
    vis = img.copy()
    cv2.polylines(vis, [corners], True, (0, 255, 255), 2)
    # Draw all edges (yellow)
    for i in range(4):
        cv2.line(vis, tuple(corners[i]), tuple(corners[(i + 1) % 4]), (0, 255, 255), 2)

    # Draw corners as circles (red)
    for pt in corners:
        cv2.circle(vis, tuple(pt), 8, (0, 0, 255), -1)

    # Draw longest edge in green
    if width > height:
        cv2.line(vis, tuple(tl), tuple(tr), (0, 255, 0), 3)
        cx, cy = (tl[0] + tr[0]) // 2, (tl[1] + tr[1]) // 2
    else:
        cv2.line(vis, tuple(tl), tuple(bl), (0, 255, 0), 3)
        cx, cy = (tl[0] + bl[0]) // 2, (tl[1] + bl[1]) // 2

    # Label the longest edge
    cv2.putText(vis, f"{real_length_mm} mm", (cx, cy),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Ruler image with 5mm Grid Overlay
    step_px = max(1, round(5.0 / pixel_spacing))
    grid = vis.copy()
    for x in range(0, w, step_px):
        cv2.line(grid, (x, 0), (x, h), (255, 0, 0), 1)
    for y in range(0, h, step_px):
        cv2.line(grid, (0, y), (w, y), (255, 0, 0), 1)

    vis_grid = cv2.addWeighted(grid, 0.4, vis, 0.6, 0)

    return pixel_spacing, vis, vis_grid


In [ ]:
def overlay_mask_simple(image, mask, color=[255, 0, 0], alpha=0.5):
    """
    Overlay a binary mask on an image.

    Args:
        image (np.ndarray): H x W x 3 RGB image
        mask (np.ndarray): H x W binary mask (1=foreground, 0=background)
        color (list): RGB color for the mask overlay
        alpha (float): transparency factor (0=transparent, 1=opaque)

    Returns:
        np.ndarray: Image with mask overlay
    """
    overlay = image.copy()

    # Create a color mask of same shape as image
    colored_mask = np.zeros_like(image, dtype=np.uint8)
    colored_mask[mask == 1] = color

    # Blend the image and the color mask
    output = cv2.addWeighted(overlay, 1.0, colored_mask, alpha, 0)

    return output

In [ ]:
def visualize_mask_overlay(pred_mask, gt_mask, resize_gt=True):
    """
    Visualizes the pixel-wise differences between predicted and ground truth masks.

    Colors:
        Yellow = overlap (true positive)
        Red    = prediction only (false positive)
        Green  = GT only (false negative)
    """

    # Ensure masks are binary [0, 255]
    pred_bin = (pred_mask > 0).astype(np.uint8) * 255
    gt_bin = (gt_mask > 0).astype(np.uint8) * 255

    # Resize GT to match prediction
    if resize_gt and pred_bin.shape != gt_bin.shape:
        gt_bin = cv2.resize(gt_bin,
                            (pred_bin.shape[1], pred_bin.shape[0]),
                            interpolation=cv2.INTER_NEAREST)

    # Create color output
    overlay = np.zeros((pred_bin.shape[0], pred_bin.shape[1], 3), dtype=np.uint8)

    # Yellow = correct prediction (intersection)
    overlay[(pred_bin == 255) & (gt_bin == 255)] = (0, 255, 255)

    # Red = prediction only (FP)
    overlay[(pred_bin == 255) & (gt_bin == 0)] = (0, 0, 255)

    # Green = GT only (FN)
    overlay[(pred_bin == 0) & (gt_bin == 255)] = (0, 255, 0)

    return overlay


In [ ]:
def visualize_mask_on_wound_with_grid(wound_path, pred_mask, gt_mask, original_pixel_spacing, alpha=0.6):
    # 1. Load the original wound image
    wound_img = cv2.imread(wound_path)
    if wound_img is None:
        raise FileNotFoundError(f"Cannot load wound image: {wound_path}")

    # Original dimensions
    original_h, original_w = wound_img.shape[:2]

    # Target dimensions (from the mask)
    target_h, target_w = pred_mask.shape

    # 2. Resizing the Wound Image (if necessary)
    if (original_h, original_w) != pred_mask.shape:
         wound_img = cv2.resize(wound_img, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

    # Current dimensions (used for drawing the grid lines later)
    h, w = wound_img.shape[:2]


    # *** CRITICAL FIX: SCALE THE PIXEL SPACING ***

    # 3. Calculate the Resizing Factor (assuming uniform scaling)
    resize_factor = target_w / original_w

    # 4. Calculate the Scaled Pixel Spacing (mm/px for the displayed image)
    # The new spacing is smaller (fewer mm per pixel) because the image is larger.
    scaled_pixel_spacing = original_pixel_spacing / resize_factor


    # 5. Generate the color-coded mask visualization (Red/Green/Yellow)
    color_mask = visualize_mask_overlay(pred_mask, gt_mask, resize_gt=False)

    # 6. Fixed Blending (same logic as before to prevent darkening)
    blended_img = wound_img.copy()
    mask_area = np.any(color_mask > 0, axis=2)
    original_float = blended_img.astype(np.float32)
    color_float = color_mask.astype(np.float32)
    original_float[mask_area] = (original_float[mask_area] * (1.0 - alpha)) + \
                                (color_float[mask_area] * alpha)
    blended_img = original_float.astype(np.uint8)


    # 7. Draw the 5mm grid using the SCALED pixel spacing
    # step_px is now accurate for the resized image
    step_px = max(1, round(5.0 / scaled_pixel_spacing))

    grid_color = (255, 255, 255)
    grid_thickness = 1

    # Draw vertical lines
    for x in range(0, w, step_px):
        cv2.line(blended_img, (x, 0), (x, h), grid_color, grid_thickness)

    # Draw horizontal lines
    for y in range(0, h, step_px):
        cv2.line(blended_img, (0, y), (w, y), grid_color, grid_thickness)

    return blended_img

In [ ]:
def visualize_mask_on_wound_with_grid_v2(wound_path, pred_mask, gt_mask, original_pixel_spacing, alpha=0.6):
    # 1. Load the original wound image
    wound_img = cv2.imread(wound_path)
    if wound_img is None:
        raise FileNotFoundError(f"Cannot load wound image: {wound_path}")

    # Original dimensions
    original_h, original_w = wound_img.shape[:2]

    # Target dimensions (from the mask)
    target_h, target_w = pred_mask.shape

    # 2. Resizing the Wound Image (if necessary)
    if (original_h, original_w) != pred_mask.shape:
         wound_img = cv2.resize(wound_img, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

    # Current dimensions (used for drawing the grid lines later)
    h, w = wound_img.shape[:2]

    # 3. Calculate the Resizing Factor (assuming uniform scaling)
    scale_x = target_w / original_w
    scale_y = target_h / original_h

    if isinstance(original_pixel_spacing, (tuple, list)):
      spacing_x_raw = original_pixel_spacing[0]
      spacing_y_raw = original_pixel_spacing[1]
    else:
      spacing_x_raw = spacing_y_raw = original_pixel_spacing

    # 4. Calculate the Scaled Pixel Spacing (mm/px for the displayed image)
    # The new spacing is smaller (fewer mm per pixel) because the image is larger.
    final_spacing_x = spacing_x_raw / scale_x
    final_spacing_y = spacing_y_raw / scale_y


    # 5. Generate the color-coded mask visualization (Red/Green/Yellow)
    color_mask = visualize_mask_overlay(pred_mask, gt_mask, resize_gt=False)

    # 6. Fixed Blending (same logic as before to prevent darkening)
    blended_img = wound_img.copy()
    mask_area = np.any(color_mask > 0, axis=2)
    original_float = blended_img.astype(np.float32)
    color_float = color_mask.astype(np.float32)
    original_float[mask_area] = (original_float[mask_area] * (1.0 - alpha)) + \
                                (color_float[mask_area] * alpha)
    blended_img = original_float.astype(np.uint8)


    # 7. Draw the 5mm grid using the SCALED pixel spacing
    # step_px is now accurate for the resized image
    step_x = max(1, round(5.0 / final_spacing_x))
    step_y = max(1, round(5.0 / final_spacing_y))

    grid_color = (255, 255, 255)
    grid_thickness = 1

    # Draw vertical lines
    for x in range(0, w, step_x):
        cv2.line(blended_img, (x, 0), (x, h), grid_color, grid_thickness)

    # Draw horizontal lines
    for y in range(0, h, step_y):
        cv2.line(blended_img, (0, y), (w, y), grid_color, grid_thickness)

    return blended_img

In [ ]:
def process_masks_and_metrics(wound_data_list, spacing_lookup, target_size=(2560, 1920)):

    results = []

    for i, data in enumerate(wound_data_list):
        wound_path = data['wound_path']
        gt_path = data['gt_path']
        predicted_path = data['predicted_path']

        # Extract wound_id from the path or the dictionary key
        # Assuming you've added 'wound_id' to the data dictionary in get_wound_predicted_gt_paths
        wound_id = data.get('wound_id')

        print(f"\n--- Processing {wound_id} ({i+1}/{len(wound_data_list)}) ---")

        try:
            # --- GET AND CONVERT PIXEL SPACING ---
            single_pixel_spacing = spacing_lookup.get(wound_id)
            if single_pixel_spacing is None:
                raise ValueError(f"Pixel spacing not found for {wound_id}")

            # Convert the single float to the required tuple format: (dx, dy)
            pixel_spacing_tuple = (single_pixel_spacing, single_pixel_spacing)
            print(f"  Pixel Spacing (mm/px): {pixel_spacing_tuple}")


            # 1. READ & RESIZE MASKS (omitted details for brevity, same as previous answer)
            gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
            pred_mask = cv2.imread(predicted_path, cv2.IMREAD_GRAYSCALE)

            # ... (Error checking and thresholding steps) ...

            gt_mask_resized = cv2.resize(gt_mask, target_size, interpolation=cv2.INTER_NEAREST)
            pred_mask_resized = cv2.resize(pred_mask, target_size, interpolation=cv2.INTER_NEAREST)


            # 2. CALCULATE METRICS (PASSING THE TUPLE)
            hd = hausdorff_distance_optimized(pred_mask_resized, gt_mask_resized, pixel_spacing_tuple)
            assd = assd_optimized(pred_mask_resized, gt_mask_resized, pixel_spacing_tuple)

            print(f"  Hausdorff Distance (mm): {hd:.3f}")
            print(f"ASSD (mm): {assd:.3f}")

            # --- FIX: Capture the returned image and display it ---
            # overlay_img = visualize_mask_overlay(pred_mask_resized, gt_mask_resized)
            overlay_img = visualize_mask_on_wound_with_grid_v2(
                wound_path=wound_path,
                pred_mask=pred_mask_resized,
                gt_mask=gt_mask_resized,
                original_pixel_spacing=pixel_spacing_tuple,
                alpha=0.6 # Adjust transparency as needed
            )

            # OPTIONAL: Add text labels for context (as shown in a previous answer)
            cv2.putText(overlay_img, f"{wound_id} | HD: {hd:.2f}mm | ASSD: {assd:.2f}mm", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

            # --- KEY FIX: Display the image using cv2_imshow ---
            cv2_imshow(overlay_img)

            # Store results (if needed)
            results.append({
                'wound_id': wound_id,
                'HD_mm': hd,
                'ASSD_mm': assd,
                'overlay_image': overlay_img
            })


        except Exception as e:
            print(f"  An error occurred for {wound_id}: {e}")

    return results

## Test


**Note:**
- Replace your own paths that contain original wound images & the reference objects
- Prepare the `wound_list` by listing all the real world dimensions represent by the reference objects for each wound image

In [ ]:
wound_base_path = "/content/drive/MyDrive/FYP/Datasets/Test/test"
ruler_base_path = "/content/drive/MyDrive/FYP/Datasets/Test/test/reference_object"

In [ ]:
wound_list = [
    {"002": 90},
    {"003": 60},
    {"007": 80},
    {"008": 88},
    {"010": 100},
    {"012": 50},
    {"014": 50},
    {"019": 40},
    {"021": 50},
    {"027": 50},
    {"028": 30},
    {"029": 100},
    {"032": 50},
    {"037": 70},
    {"039": 100},
    {"040": 70},
    {"041": 100},
    {"042": 50},
    {"043": 100},
    {"045": 80},
    {"046": 70},
    {"047": 120},
    {"048": 100},
    {"049": 100},
    {"050": 40},
]

In [ ]:
wound_data = get_wound_predicted_gt_cleaned_paths(wound_base_path, wound_list)
ruler_data = get_ruler_paths(ruler_base_path, wound_list)

In [ ]:
calibration_results = []

print("Starting ruler calibration...")

for data in ruler_data:
    wound_id = data['wound_id']
    ruler_path = data['ruler_path']
    real_length_mm = data['real_length_mm']

    try:
        # Call your calibration function
        pixel_spacing, vis_img, vis_grid_img = calibrate_ruler_convex_hull_grid(
            ruler_path, real_length_mm
        )

        # Store the results
        calibration_results.append({
            'wound_id': wound_id,
            'ruler_path': ruler_path,
            'real_length_mm': real_length_mm,
            'pixel_spacing': pixel_spacing,
            'visualization_image': vis_img,      # The image with corners/longest edge
            'grid_overlay_image': vis_grid_img   # The image with the 5mm grid
        })
        print(f"✅ Success for {ruler_path}")

    except (FileNotFoundError, ValueError) as e:
        print(f"❌ Failed to process {ruler_path}: {e}")

print("Calibration finished.")

In [ ]:
for i, result in enumerate(calibration_results):
    # 1. Extract the image and its metadata
    vis_grid = result['grid_overlay_image']
    real_length = result['real_length_mm']
    ruler_path = result['ruler_path']

    # 2. Convert from BGR (OpenCV format) to RGB (Matplotlib format)
    # This is necessary for correct color display in Matplotlib.
    vis_grid_rgb = cv2.cvtColor(vis_grid, cv2.COLOR_BGR2RGB)

    # 3. Display the image
    plt.figure(figsize=(20, 20))
    plt.imshow(vis_grid_rgb)

    # Create a clear title
    title = f"Ruler {i+1}: ID {ruler_path.split('/')[-1].split('_')[0]} | Real Length: {real_length} mm"
    plt.title(title, fontsize=14)
    plt.axis('off') # Hide axes for a clean display
    plt.show()

In [ ]:
# Create a dictionary for quick lookup of pixel spacing
# Mapping: 'wound_id' -> pixel_spacing_float
spacing_lookup = {
    result['wound_id']: result['pixel_spacing']
    for result in calibration_results
}

In [ ]:
all_results = process_masks_and_metrics(wound_data, spacing_lookup)